<a href="https://colab.research.google.com/github/Bazinga97/Recommendation-system-/blob/main/Amazon_Electronics_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Problem Statement **

Online E-commerce websites like Amazon, Filpkart uses different recommendation models to provide different suggestions to different users. Build your own recommendation system for products on an e-commerce website like Amazon.com.

Amazon: ** Collaborative filtering**: predict user preference by collecting preferences from other users

**Assumptions**: Two different users: having same preferences : then they most likely same preferences over other items

## Types: User_based and item_based

**User_based**: recommends items by fiding similar users

**Item_based**: similar items based on user interactions and recommends similar to ones the user has already liked


Scales: data sets and produces high-quality recommendations in real time.

Filtering matches each of the user's purchases and rated items to similar items, : recommendation list for the user.



Dataset
The source of the dataset - Amazon Reviews data (http://jmcauley.ucsd.edu/data/amazon/). The repository has several datasets. For this case study, we are using the Electronics dataset. The columns in this dataset are:

userId

productId

ratings

timestamp


In [ ]:
## Basic imports

import math, random, warnings  ## for math module / warning controls over behaviour of warning messages in python
from time import time
from datetime import datetime
from collections import defaultdict ## create dictionaries with default values for missing keys
from IPython.core.interactiveshell import InteractiveShell ## interactive python interpreter by modifying appearence of shell by incoporating custom commands

In [ ]:
import numpy as np ## import numpy library for multidimensional array and matrices
from scipy.sparse.linalg import svds  ## Singular value decomposition(SVD): decomposes a matrix into three constituent matrices for dimensionality reduction and capturing latent factors
from sklearn import model_selection ## Module includes: model selection and evalutaion = Cross-validation, train-test split and hyperparamter tuning
from sklearn.metrics.pairwise import cosine_similarity # Cosine similarity measures: similarity between two vectors in multi-dimensional space employed in recommendation and text filtering

In [ ]:
# Data handling
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

Scikit-learn / Scikit-surprise

Scikit-learn : used for supervised and unsupervised learning for classification, clustering and dimensionality reduction


Scikit-surprise: particularly for design and building and analysing recommendation tasks

Scikit-surprise focuses : collaborative filtering techniques,

Matrix factorization,
Neighborhood-based methods,
Baseline models.

Simplify the process by  evaluating recommendation algorithms, with support for dataset handling, cross-validation, and hyperparameter tuning.


In [ ]:
from surprise import SVD, KNNWithMeans
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import train_test_split, GridSearchCV
from surprise.prediction_algorithms.baseline_only import BaselineOnly


In [ ]:
# Configure for any default setting of any library
InteractiveShell.ast_node_interactivity = "all"
warnings.filterwarnings('ignore')
%matplotlib inline
# sns.set(style='whitegrid', palette='deep', font='sans-serif', font_scale=1.2, color_codes=True)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Exploratory data analysis

In [ ]:

# Load the dataset into a Pandas dataframe called ratingsand skip any lines that return an error
ratings = pd.read_csv('ratings_Electronics(1).csv',
                      names=['userId', 'productId', 'rating', 'timestamp'],
                      error_bad_lines=False,
                      warn_bad_lines=False)

# Save an original copy of the dataframe
ratings_original = ratings.copy(deep=True)

TypeError: read_csv() got an unexpected keyword argument 'error_bad_lines'